# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning.

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.


In [7]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)
        


names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()


,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*


In [8]:
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero()


# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*


In [9]:
import numpy as np
def answer_one():
    max_gold = np.max(df['Gold'])
    df.index.name = 'Country'
    res_df = df[df['Gold'] == max_gold]
    return res_df.iloc[0].name


### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*


In [10]:
def answer_two():
    series_df = df['Gold'] - df['Gold.1']
    series_df = abs(series_df)
    filtered_series = series_df[series_df == np.max(series_df)]
    return filtered_series.index.tolist()[0]


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*


In [11]:
def answer_three():
    more_than_one = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    fil_df = abs(more_than_one['Gold'] - more_than_one['Gold.1']) / more_than_one['Gold.2']
    return fil_df[fil_df == np.max(fil_df)].index.tolist()[0]


### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*


In [12]:
def answer_four():
    temp_df = df.copy()
    temp_df['Gold.2'] *= 3
    gold_df = temp_df['Gold.2']
    temp_df['Silver.2'] *= 2
    silver_df = temp_df['Silver.2']
    temp_df['Bronze.2'] *= 2
    bronze_df = temp_df['Bronze.2']

    res_df = gold_df + silver_df + bronze_df
    res_df.name = 'Points'
    return res_df


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*


In [17]:
census_df = pd.read_csv('census.csv')
def answer_five():
    unique_states = census_df['STNAME'].unique()
    census_df_1 = census_df.set_index(['STNAME','CTYNAME'])
    maxi = -32768
    for each_state in unique_states:
        no = census_df_1.loc[each_state]
        curr = len(no.index)
        if (curr > maxi):
            maxi = curr
            final_state = each_state
    return final_state


### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*


In [18]:
def answer_six():
    unique_states = census_df['STNAME'].unique()
    census_df_2 = census_df.set_index(['STNAME','CTYNAME'])
    final_dict = {}
    for each_state in unique_states:
        census_df_3 = census_df_2.loc[each_state]
        census_df_3 = census_df_3['CENSUS2010POP']
        census_df_3 = census_df_3.sort_values(ascending=False)
        census_df_3 = census_df_3.fillna(0)
        if (len(census_df_3.index) >= 4):
            curr = census_df_3.iloc[1] + census_df_3.iloc[2] + census_df_3.iloc[3]
        elif (len(census_df_3.index) == 3):
            curr = census_df_3.iloc[1] + census_df_3.iloc[2]
        elif (len(census_df_3.index) == 2):
            curr = census_df_3.iloc[1]
        final_dict[each_state] = curr
    census_df_4 = pd.Series(final_dict)
    census_df_4 = census_df_4.sort_values(ascending=False)

    res_list = census_df_4.index.tolist()[0:3]
    return res_list


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*


In [19]:
#WRONG ANSWER

def answer_seven():
    unique_counties = census_df['CTYNAME'].unique()
    census_df_5 = census_df.set_index(['CTYNAME'])
    final_dict = {}
    for each_county in unique_counties:
        census_df_6 = census_df_5.loc[each_county]
        if (len(census_df_6.index) == 1) and census_df_6['STNAME'] == each_county:
            census_df_6.drop(each_county)
        elif (len(census_df_6.index) > 1):
            census_df_6[census_df_6['STNAME'] != each_county]
            
            #census_df_6[census_df_6['STNAME'] == each_county]
            census_df_6 = census_df_6.fillna(0)
            sum1 = np.sum(census_df_6['POPESTIMATE2010'])
            sum2 = np.sum(census_df_6['POPESTIMATE2011'])
            sum3 = np.sum(census_df_6['POPESTIMATE2012'])
            sum4 = np.sum(census_df_6['POPESTIMATE2013'])
            sum5 = np.sum(census_df_6['POPESTIMATE2014'])
            sum6 = np.sum(census_df_6['POPESTIMATE2015'])
            list1 = []
            list1.extend((sum1,sum2,sum3,sum4,sum5,sum6))
            curr = max(list1) - min(list1)
            final_dict[each_county] = curr

    census_df_7 = pd.Series(final_dict)
    census_df_7 = census_df_7.sort_values(ascending=False)
    return census_df_7.index.tolist()[0]


### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*


In [20]:
def answer_eight():
    unique_counties = census_df['CTYNAME'].unique()
    census_df_8 = census_df.set_index(['CTYNAME'])
    res_df = pd.DataFrame(['STNAME','CTYNAME'])

    for each_county in unique_counties:
        census_df_9 = census_df_8.loc[each_county]

        if isinstance(census_df_9,pd.DataFrame):

            unique_states = census_df_9['STNAME'].unique()
            census_df_9 = census_df_9.set_index('STNAME')
            for each_state in unique_states:
                census_df_10 = census_df_9.loc[each_state]
                if isinstance(census_df_10,pd.DataFrame):

                    for simple in range (0,2):
                        census_df_11 = census_df_10.iloc[simple]
                        list1 = []
                        yes_or_no = (census_df_11['REGION'] == 1 or census_df_11['REGION'] == 2) and (each_county[0:10] == 'Washington') and (census_df_11['POPESTIMATE2015'] > census_df_11['POPESTIMATE2014'])
                        if yes_or_no == True:
                            list1.extend((census_df_11.name,census_df_11['CTYNAME']))
                            temp_df = pd.Series(list1)
                            res_df = res_df.append(temp_df, ignore_index=True)
                else:
                    list1 = []
                    yes_or_no = (census_df_10['REGION'] == 1 or census_df_10['REGION'] == 2) and (each_county[0:10] == 'Washington') and (census_df_10['POPESTIMATE2015'] > census_df_10['POPESTIMATE2014'])
                    if yes_or_no == True:
                        #print (census_df_10)

                        list1.extend((census_df_10.name,each_county))
                        temp_df = pd.Series(list1)
                        res_df = res_df.append(temp_df, ignore_index=True)


        else:
            yes_or_no = (census_df_9['REGION'] == 1 or census_df_9['REGION'] == 2) and (each_county[0:10] == 'Washington') and (census_df_9['POPESTIMATE2015'] > census_df_9['POPESTIMATE2014'])
            list2 = []
            if yes_or_no == True:
                list2.extend((census_df_9['STNAME'],census_df_9.name))
                temp_df = pd.Series(list2)
                print (temp_df)
                res_df = res_df.append(temp_df, ignore_index=True)

    res_df = res_df.drop([0,1])
    res_df = res_df.reset_index(drop=True)
    res_df.columns = ['STNAME','CTYNAME']
    return res_df
